In [1]:
import pandas
import xarray

df = pandas.read_csv(
    "../pathfinder2e_stats/_tables/SIMPLE_PC/classes.csv", index_col=[0], header=[0, 1]
).fillna("")
df

SIMPLE_PC_table                  strike_bonus                     \
PC_table                 ability_bonus.boosts ability_bonus.apex   
alchemist/bomber                            3               True   
alchemist/chirurgeon                        3               True   
alchemist/mutagenist                        3               True   
alchemist/toxicologist                      3               True   
animist                                     3              False   
barbarian                                   4               True   
bard                                        3              False   
champion                                    4               True   
cleric/battle creed                         3               True   
cleric/cloistered cleric                    3              False   
cleric/warpriest                            3              False   
commander                                   4               True   
druid                                       3              False   
exemplar                                    4               True   
fighter                                     4               True   
guardian                                    4               True   
gunslinger/DEX                              4               True   
gunslinger/STR                              3              False   
inventor                                    3               True   
investigator                                3               True   
kineticist                                  4               True   
magus                                       4               True   
monk                                        4               True   
oracle                                      3              False   
psychic                                     3              False   
ranger                                      4               True   
rogue                                       4               True   
sorcerer                                    3              False   
summoner                                    4              False   
swashbuckler                                4               True   
thaumaturge                                 4               True   
witch                                       3              False   
wizard                                      3              False   

SIMPLE_PC_table                                                \
PC_table                 attack_item_bonus weapon_proficiency   
alchemist/bomber                      bomb               True   
alchemist/chirurgeon          potency_rune               True   
alchemist/mutagenist       bestial_mutagen               True   
alchemist/toxicologist        potency_rune               True   
animist                       potency_rune               True   
barbarian                     potency_rune               True   
bard                          potency_rune               True   
champion                      potency_rune               True   
cleric/battle creed           potency_rune               True   
cleric/cloistered cleric      potency_rune               True   
cleric/warpriest              potency_rune               True   
commander                     potency_rune               True   
druid                         potency_rune               True   
exemplar                      potency_rune               True   
fighter                       potency_rune               True   
guardian                      potency_rune               True   
gunslinger/DEX                potency_rune               True   
gunslinger/STR                potency_rune               True   
inventor                      potency_rune               True   
investigator                  potency_rune               True   
kineticist                 gate_attenuator              False   
magus                         potency_rune               True   
monk                          potency_rune               Tr

In [2]:
import pathfinder2e_stats as pf2

PC = pf2.tables.PC

In [3]:
def _class_name(df):
    return xarray.DataArray(df.index.str.replace(r"/.*", "", regex=True)).rename(
        {"dim_0": "class"}
    )


def _get_ability_boosts(df):
    return (
        pf2.tables.PC.ability_bonus.boosts.sel(
            initial=df["ability_bonus.boosts"].values
        )
        .rename({"initial": "class"})
        .drop_vars("class")
    )


def _get_ability_apex(df):
    return pf2.tables.PC.ability_bonus.apex * xarray.DataArray(
        df["ability_bonus.apex"].values, dims=["class"]
    )


def _get_proficiency(df, prof_name):
    prof = pf2.tables.PC[prof_name].to_array("class").sel({"class": _class_name(df)})
    mask = xarray.DataArray(df[prof_name].values, dims=["class"])
    return prof * mask


def _merge_components(components: dict[str, xarray.DataArray]) -> xarray.Dataset:
    ds = xarray.Dataset(components)
    rows = []

    ds_other = ds
    for class_name, dim in (
        ("alchemist", "research_field"),
        ("cleric", "_tmp_doctrine2"),
        ("gunslinger", "ability"),
    ):
        ds_i = ds.sel({"class": ds["class"] == class_name})
        ds_i = ds_i.rename({"class": dim})
        ds_i[dim] = [
            v.split("/")[1]
            for v in df.index.values.tolist()
            if v.startswith(class_name + "/")
        ]
        ds_i = ds_i.expand_dims({"class": [class_name]})
        ds_other = ds_other.sel({"class": ds_other["class"] != class_name})
        rows.append(ds_i)
    rows.append(ds_other)
    ds = xarray.concat(rows, dim="class").to_array("component").to_dataset("class")

    vars = {}
    for class_name, da in ds.data_vars.items():
        if class_name == "cleric":
            assert (da._tmp_doctrine2.values == da.doctrine.values).all()
            da = da.sel(_tmp_doctrine2=da.doctrine)
            del da.coords["_tmp_doctrine2"]

        for dim in list(da.dims):
            if dim not in ("level", "component") and (da.isel({dim: 0}) == da).all():
                da = da.isel({dim: 0}, drop=True)

        da = xarray.concat(
            [da.level.expand_dims({"component": ["level"]}), da],
            dim="component",
        )
        vars[class_name] = da

    ds = xarray.Dataset(dict(sorted(vars.items())))

    if "mastery" in ds.dims:
        return ds.transpose("level", "component", "mastery", ...)
    return ds.transpose("level", "component", ...)

In [4]:
df_strike = df["strike_bonus"]

components = {}
components["ability_boosts"] = _get_ability_boosts(df_strike)
components["ability_apex"] = _get_ability_apex(df_strike)

components["item"] = (
    pf2.tables.PC.attack_item_bonus.to_array("variable")
    .sel(variable=df_strike["attack_item_bonus"].values)
    .T.rename({"variable": "class"})
    .drop_vars("class")
)

for prof_name in ("weapon_proficiency", "class_proficiency"):
    components[prof_name] = _get_proficiency(df_strike, prof_name)

ds_strike = _merge_components(components)

In [5]:
ds_strike

<xarray.Dataset> Size: 35kB
Dimensions:         (component: 6, level: 20, research_field: 4, doctrine: 3,
                     mastery: 2, ability: 2)
Coordinates:
  * component       (component) object 48B 'level' ... 'class_proficiency'
  * level           (level) int64 160B 1 2 3 4 5 6 7 8 ... 14 15 16 17 18 19 20
  * research_field  (research_field) <U12 192B 'bomber' ... 'toxicologist'
  * doctrine        (doctrine) <U17 204B 'battle creed' ... 'warpriest'
  * mastery         (mastery) bool 2B True False
  * ability         (ability) <U3 24B 'DEX' 'STR'
Data variables: (12/27)
    alchemist       (level, component, research_field) int64 4kB 1 1 1 ... 0 0 0
    animist         (level, component) int64 960B 1 3 0 0 2 0 2 ... 20 5 0 3 4 0
    barbarian       (level, component) int64 960B 1 4 0 0 2 0 2 ... 20 6 1 3 6 0
    bard            (level, component) int64 960B 1 3 0 0 2 0 2 ... 20 5 0 3 4 0
    champion        (level, component) int64 960B 1 4 0 0 2 0 2 ... 20 6 1 3 6 0
    cleric          (level, component, doctrine) int64 3kB 1 1 1 3 3 ... 6 0 0 0
    ...              ...
    sorcerer        (level, component) int64 960B 1 3 0 0 2 0 2 ... 20 5 0 3 4 0
    summoner        (level, component) int64 960B 1 4 0 0 2 0 2 ... 20 6 0 3 6 0
    swashbuckler    (level, component) int64 960B 1 4 0 0 2 0 2 ... 20 6 1 3 6 0
    thaumaturge     (level, component) int64 960B 1 4 0 0 2 0 2 ... 20 6 1 3 6 0
    witch           (level, component) int64 960B 1 3 0 0 2 0 2 ... 20 5 0 3 4 0
    wizard          (level, component) int64 960B 1 3 0 0 2 0 2 ... 20 5 0 3 4 0

In [6]:
ds_strike.sum("component").display()

variable 
 alchemist 
 animist 
 barbarian 
 bard 
 champion 
 cleric 
 commander 
 druid 
 exemplar 
 fighter 
 guardian 
 gunslinger 
 inventor 
 investigator 
 kineticist 
 magus 
 monk 
 oracle 
 psychic 
 ranger 
 rogue 
 sorcerer 
 summoner 
 swashbuckler 
 thaumaturge 
 witch 
 wizard 
 
 
 research_field 
 bomber 
 chirurgeon 
 mutagenist 
 toxicologist 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 doctrine 
 
 
 
 
 
 
 
 
 battle creed 
 cloistered cleric 
 warpriest 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 mastery 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 True 
 False 
 
 True 
 False 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ability 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 DEX 
 STR 
 DEX 
 STR 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 level 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 6 
 6 
 7 
 6 
 6 
 7 
 6 
 7 
 6 
 6 
 6 
 7 
 6 
 7 
 9 
 9 
 7 
 9 
 8 
 7 
 6 
 6 
 6 
 7 
 7 
 7 
 6 
 6 
 7 
 7 
 6 
 7 
 7 
 7 
 6 
 6 
 
 
 2 
 7 
 8 
 8 
 8 
 8 
 9 
 8 
 9 
 8 
 8 
 8 
 9 
 8 
 9 
 11 
 11 
 9 
 11 
 10 
 9 
 8 
 8 
 8 
 8 
 9 
 9 
 8 
 8 
 9 
 9 
 8 
 9 
 9 
 9 
 8 
 8 
 
 
 3 
 9 
 9 
 10 
 9 
 9 
 10 
 9 
 10 
 9 
 9 
 9 
 10 
 9 
 10 
 12 
 12 
 10 
 12 
 11 
 10 
 9 
 9 
 9 
 10 
 10 
 10 
 9 
 9 
 10 
 10 
 9 
 10 
 10 
 10 
 9 
 9 
 
 
 4 
 10 
 10 
 11 
 10 
 10 
 11 
 10 
 11 
 10 
 10 
 10 
 11 
 10 
 11 
 13 
 13 
 11 
 13 
 12 
 11 
 10 
 10 
 10 
 11 
 11 
 11 
 10 
 10 
 11 
 11 
 10 
 11 
 11 
 11 
 10 
 10 
 
 
 5 
 12 
 12 
 13 
 12 
 12 
 14 
 12 
 14 
 14 
 12 
 12 
 14 
 12 
 14 
 16 
 14 
 14 
 16 
 16 
 14 
 14 
 14 
 14 
 12 
 14 
 14 
 12 
 12 
 14 
 14 
 12 
 14 
 14 
 14 
 12 
 12 
 
 
 6 
 13 
 13 
 14 
 13 
 13 
 15 
 13 
 15 
 15 
 13 
 13 
 15 
 13 
 15 
 17 
 15 
 15 
 17 
 17 
 15 
 15 
 15 
 15 
 13 
 15 
 15 
 13 
 13 
 15 
 15 
 13 
 15 
 15 
 15 
 13 
 13 
 
 
 7 
 16 
 16 
 17 
 16 
 14 
 16 
 14 
 16 
 16 
 14 
 16 
 16 
 14 
 16 
 18 
 16 
 16 
 18 
 18 
 16 
 16 
 16 
 16 
 16 
 16 
 16 
 14 
 14 
 16 
 16 
 14 
 16 
 16 
 16 
 14 
 14 
 
 
 8 
 17 
 17 
 18 
 17 
 15 
 17 
 15 
 17 
 17 
 15 
 17 
 17 
 15 
 17 
 19 
 17 
 17 
 19 
 19 
 17 
 17 
 17 
 17 
 17 
 17 
 17 
 15 
 15 
 17 
 17 
 15 
 17 
 17 
 17 
 15 
 15 
 
 
 9 
 18 
 18 
 19 
 18 
 16 
 18 
 16 
 18 
 18 
 16 
 18 
 18 
 16 
 18 
 20 
 18 
 18 
 20 
 20 
 18 
 18 
 18 
 18 
 18 
 18 
 18 
 16 
 16 
 18 
 18 
 16 
 18 
 18 
 18 
 16 
 16 
 
 
 10 
 19 
 20 
 20 
 20 
 18 
 21 
 18 
 21 
 20 
 18 
 20 
 21 
 18 
 21 
 23 
 21 
 21 
 23 
 22 
 21 
 20 
 20 
 20 
 20 
 21 
 21 
 18 
 18 
 21 
 21 
 18 
 21 
 21 
 21 
 18 
 18 
 
 
 11 
 21 
 21 
 22 
 21 
 21 
 22 
 21 
 22 
 21 
 21 
 21 
 22 
 21 
 22 
 24 
 22 
 22 
 24 
 23 
 22 
 21 
 21 
 21 
 22 
 22 
 22 
 21 
 21 
 22 
 22 
 21 
 22 
 22 
 22 
 21 
 21 
 
 
 12 
 22 
 22 
 23 
 22 
 22 
 23 
 22 
 23 
 22 
 22 
 22 
 23 
 22 
 23 
 25 
 23 
 23 
 25 
 24 
 23 
 22 
 22 
 22 
 23 
 23 
 23 
 22 
 22 
 23 
 23 
 22 
 23 
 23 
 23 
 22 
 22 
 
 
 13 
 23 
 23 
 24 
 23 
 23 
 26 
 23 
 26 
 25 
 23 
 23 
 26 
 23 
 26 
 28 
 26 
 26 
 28 
 27 
 26 
 25 
 25 
 25 
 24 
 26 
 26 
 23 
 23 
 26 
 26 
 23 
 26 
 26 
 26 
 23 
 23 
 
 
 14 
 24 
 24 
 25 
 24 
 24 
 27 
 24 
 27 
 26 
 24 
 24 
 27 
 24 
 27 
 29 
 27 
 27 
 29 
 28 
 27 
 26 
 26 
 26 
 25 
 27 
 27 
 24 
 24 
 27 
 27 
 24 
 27 
 27 
 27 
 24 
 24 
 
 
 15 
 26 
 26 
 27 
 26 
 26 
 28 
 26 
 28 
 28 
 26 
 26 
 28 
 26 
 28 
 30 
 28 
 28 
 30 
 30 
 28 
 28 
 28 
 28 
 28 
 28 
 28 
 26 
 26 
 28 
 28 
 26 
 28 
 28 
 28 
 26 
 26 
 
 
 16 
 27 
 28 
 28 
 28 
 28 
 30 
 28 
 30 
 30 
 28 
 28 
 30 
 28 
 30 
 32 
 30 
 30 
 32 
 32 
 30 
 30 
 30 
 30 
 29 
 30 
 30 
 28 
 28 
 30 
 30 
 28 
 30 
 30 
 30 
 28 
 28 
 
 
 17 
 30 
 30 
 31 
 30 
 29 
 32 
 29 
 32 
 32 
 29 
 29 
 32 
 29 
 32 
 34 
 32 
 32 
 34 
 33 
 32 
 31 
 32 
 32 
 32 
 32 
 32 
 29 
 29 
 32 
 32 
 29 
 31 
 32 
 32 
 29 
 29 
 
 
 18 
 31 
 31 
 32 
 31 
 30 
 33 
 30 
 33 
 

In [7]:
df_spell = df["spell_bonus"]

components = {}
components["ability_boosts"] = _get_ability_boosts(df_spell)
components["ability_apex"] = _get_ability_apex(df_spell)
components["spell_proficiency"] = xarray.concat(
    [
        pf2.tables.PC.spell_proficiency.to_array("class").sel(
            {"class": _class_name(df_spell[df_spell.spell_proficiency == "%class%"])}
        ),
        pf2.tables.PC.spell_proficiency.dedication
        * xarray.ones_like(
            _class_name(df_spell[df_spell.spell_proficiency == "dedication"]),
            dtype=int,
        ),
        pf2.tables.PC.spell_proficiency.dedication
        * xarray.zeros_like(
            _class_name(df_spell[df_spell.spell_proficiency == ""]),
            dtype=int,
        ),
    ],
    dim="class",
).sortby("class")
components["class_proficiency"] = _get_proficiency(df_spell, "class_proficiency")

ds_spell = _merge_components(components)

In [8]:
ds_spell

<xarray.Dataset> Size: 24kB
Dimensions:       (component: 5, level: 20, doctrine: 3)
Coordinates:
  * component     (component) object 40B 'level' ... 'class_proficiency'
  * level         (level) int64 160B 1 2 3 4 5 6 7 8 ... 13 14 15 16 17 18 19 20
  * doctrine      (doctrine) <U17 204B 'battle creed' ... 'warpriest'
Data variables: (12/27)
    alchemist     (level, component) int64 800B 1 4 0 2 0 2 4 ... 6 0 20 6 0 6 0
    animist       (level, component) int64 800B 1 4 0 2 0 2 4 ... 8 0 20 6 1 8 0
    barbarian     (level, component) int64 800B 1 3 0 2 0 2 3 ... 6 0 20 5 0 6 0
    bard          (level, component) int64 800B 1 4 0 2 0 2 4 ... 6 0 20 6 1 6 0
    champion      (level, component) int64 800B 1 3 0 2 0 2 3 ... 6 0 20 5 0 6 0
    cleric        (level, component, doctrine) int64 2kB 1 1 1 4 4 ... 8 6 0 0 0
    ...            ...
    sorcerer      (level, component) int64 800B 1 4 0 2 0 2 4 ... 8 0 20 6 1 8 0
    summoner      (level, component) int64 800B 1 4 0 2 0 2 4 ... 6 0 20 6 1 6 0
    swashbuckler  (level, component) int64 800B 1 4 0 2 0 2 4 ... 6 0 20 6 0 6 0
    thaumaturge   (level, component) int64 800B 1 3 0 2 0 2 3 ... 6 0 20 5 0 6 0
    witch         (level, component) int64 800B 1 4 0 2 0 2 4 ... 8 0 20 6 1 8 0
    wizard        (level, component) int64 800B 1 4 0 2 0 2 4 ... 8 0 20 6 1 8 0

In [9]:
ds_spell.sum("component").display()

variable 
 alchemist 
 animist 
 barbarian 
 bard 
 champion 
 cleric 
 commander 
 druid 
 exemplar 
 fighter 
 guardian 
 gunslinger 
 inventor 
 investigator 
 kineticist 
 magus 
 monk 
 oracle 
 psychic 
 ranger 
 rogue 
 sorcerer 
 summoner 
 swashbuckler 
 thaumaturge 
 witch 
 wizard 
 
 
 doctrine 
 
 
 
 
 
 battle creed 
 cloistered cleric 
 warpriest 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 level 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 7 
 7 
 6 
 7 
 6 
 7 
 7 
 7 
 6 
 7 
 6 
 6 
 6 
 6 
 7 
 7 
 7 
 6 
 6 
 7 
 7 
 6 
 6 
 7 
 7 
 7 
 6 
 7 
 7 
 
 
 2 
 8 
 8 
 7 
 8 
 7 
 8 
 8 
 8 
 7 
 8 
 7 
 7 
 7 
 7 
 8 
 8 
 8 
 7 
 7 
 8 
 8 
 7 
 7 
 8 
 8 
 8 
 7 
 8 
 8 
 
 
 3 
 9 
 9 
 8 
 9 
 8 
 9 
 9 
 9 
 8 
 9 
 8 
 8 
 8 
 8 
 9 
 9 
 9 
 8 
 8 
 9 
 9 
 8 
 8 
 9 
 9 
 9 
 8 
 9 
 9 
 
 
 4 
 10 
 10 
 9 
 10 
 9 
 10 
 10 
 10 
 9 
 10 
 9 
 9 
 9 
 9 
 10 
 10 
 10 
 9 
 9 
 10 
 10 
 9 
 9 
 10 
 10 
 10 
 9 
 10 
 10 
 
 
 5 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 11 
 
 
 6 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 12 
 
 
 7 
 13 
 15 
 13 
 13 
 13 
 13 
 15 
 13 
 13 
 15 
 13 
 13 
 13 
 13 
 13 
 13 
 15 
 13 
 13 
 15 
 15 
 13 
 13 
 15 
 13 
 13 
 13 
 15 
 15 
 
 
 8 
 14 
 16 
 14 
 14 
 14 
 14 
 16 
 14 
 14 
 16 
 14 
 14 
 14 
 14 
 14 
 14 
 16 
 14 
 14 
 16 
 16 
 14 
 14 
 16 
 14 
 14 
 14 
 16 
 16 
 
 
 9 
 15 
 17 
 15 
 15 
 15 
 15 
 17 
 15 
 15 
 17 
 15 
 15 
 15 
 15 
 15 
 15 
 17 
 17 
 15 
 17 
 17 
 15 
 15 
 17 
 17 
 15 
 15 
 17 
 17 
 
 
 10 
 17 
 19 
 16 
 17 
 16 
 17 
 19 
 17 
 16 
 19 
 16 
 16 
 16 
 16 
 17 
 17 
 19 
 18 
 16 
 19 
 19 
 16 
 16 
 19 
 19 
 17 
 16 
 19 
 19 
 
 
 11 
 18 
 20 
 17 
 18 
 17 
 20 
 20 
 20 
 17 
 20 
 17 
 17 
 17 
 17 
 18 
 18 
 20 
 19 
 17 
 20 
 20 
 17 
 17 
 20 
 20 
 18 
 17 
 20 
 20 
 
 
 12 
 21 
 21 
 20 
 21 
 20 
 21 
 21 
 21 
 20 
 21 
 20 
 20 
 20 
 20 
 21 
 21 
 21 
 20 
 20 
 21 
 21 
 20 
 20 
 21 
 21 
 21 
 20 
 21 
 21 
 
 
 13 
 22 
 22 
 21 
 22 
 21 
 22 
 22 
 22 
 21 
 22 
 21 
 21 
 21 
 21 
 22 
 22 
 22 
 21 
 21 
 22 
 22 
 21 
 21 
 22 
 22 
 22 
 21 
 22 
 22 
 
 
 14 
 23 
 23 
 22 
 23 
 22 
 23 
 23 
 23 
 22 
 23 
 22 
 22 
 22 
 22 
 23 
 23 
 23 
 22 
 22 
 23 
 23 
 22 
 22 
 23 
 23 
 23 
 22 
 23 
 23 
 
 
 15 
 24 
 26 
 24 
 24 
 24 
 24 
 26 
 24 
 24 
 26 
 24 
 24 
 24 
 24 
 24 
 24 
 26 
 24 
 24 
 26 
 26 
 24 
 24 
 26 
 24 
 24 
 24 
 26 
 26 
 
 
 16 
 25 
 27 
 25 
 25 
 25 
 25 
 27 
 25 
 25 
 27 
 25 
 25 
 25 
 25 
 25 
 25 
 27 
 25 
 25 
 27 
 27 
 25 
 25 
 27 
 25 
 25 
 25 
 27 
 27 
 
 
 17 
 26 
 29 
 26 
 27 
 26 
 26 
 29 
 27 
 26 
 29 
 26 
 26 
 26 
 26 
 26 
 26 
 29 
 28 
 26 
 29 
 29 
 26 
 26 
 29 
 29 
 26 
 26 
 29 
 29 
 
 
 18 
 29 
 30 
 29 
 30 
 29 
 27 
 30 
 28 
 29 
 30 
 29 
 29 
 29 
 29 
 29 
 29 
 30 
 29 
 29 
 30 
 30 
 29 
 29 
 30 
 30 
 29 
 29 
 30 
 30 
 
 
 19 
 30 
 33 
 30 
 31 
 30 
 28 
 33 
 31 
 30 
 33 
 30 
 30 
 30 
 30 
 30 
 30 
 33 
 30 
 30 
 33 
 33 
 30 
 30 
 33 
 31 
 30 
 30 
 33 
 33 
 
 
 20 
 32 
 35 
 31 
 33 
 31 
 30 
 35 
 33 
 31 
 35 
 31 
 31 
 31 
 31 
 32 
 32 
 35 
 31 
 31 
 35 
 35 
 31 
 31 
 35 
 33 
 32 
 31 
 35 
 35